<a href="https://colab.research.google.com/github/SamBrudell/report.clean/blob/main/0.4.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import gdown
import pandas as pd
import re
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)  # Ensures a fresh connection

# Google Drive file ID (Replace with your actual file ID)
file_id = '1pR8Jp7zOGV5OmWVa_5VuFBQlppK15Hi4'  # Replace with your actual file ID
url = f'https://drive.google.com/uc?id={file_id}'

# Download file from Google Drive
output_file_path = 'downloaded_file.csv'
gdown.download(url, output_file_path, quiet=False)
print(f"File downloaded to: {output_file_path}")

# Read the CSV file with encoding handling
try:
    df = pd.read_csv(output_file_path, encoding='ISO-8859-1')  # Try ISO-8859-1 encoding first
    print("File read successfully!")
except UnicodeDecodeError:
    print(f"Error reading the file with 'ISO-8859-1' encoding, trying 'utf-16' encoding...")
    df = pd.read_csv(output_file_path, encoding='utf-16')

# Check if 'Cost Centre' column exists
if 'Cost Centre' not in df.columns:
    print(f"Error: Column 'Cost Centre' not found in the dataset!")
    print("Available columns:", df.columns.tolist())  # Debugging step
else:
    print(f"'Cost Centre' column exists! Proceeding with splitting...")

# List of columns to delete (if they exist)
columns_to_delete = [
    'Entity', 'GL Date', 'Intercompany', 'Source', 'Category', 'Event Class',
    'Journal Batch', 'Journal', 'Currency', 'Currency Amount', 'Total Value'
]
df = df.drop(columns=[col for col in columns_to_delete if col in df.columns], errors='ignore')

# Function to extract the numeric value from a string (text format)
def extract_numeric_value(text):
    match = re.search(r'(\d+\.?\d*)', str(text))
    return float(match.group(1)) if match else None

# If 'Account' column exists, extract numeric values and keep specific numbers
if 'Account' in df.columns:
    df['Account_numeric'] = df['Account'].apply(extract_numeric_value)

    # List of specific numbers to keep
    keep_numbers = [
        1518, 2410, 2646, 2689, 2681, 2682, 2341, 3092, 2388, 2322, 2644, 2412,
        2391, 2841, 2688, 2852, 2385, 2321, 2384, 2352, 2683, 2735, 2405, 2377,
        2374, 2362, 2823, 2685, 2643, 2373, 2733, 2351, 2371, 3064, 2342, 2379,
        2731, 2851, 2762, 2642, 2411, 2425, 2361, 2858, 2418, 2842, 2421, 2684,
        2737, 2736, 2381, 2413, 2734, 2952, 2822, 2378, 2380, 2430, 2395, 1519,
        2404
    ]

    # Keep only rows where 'Account_numeric' is in keep_numbers
    df = df[df['Account_numeric'].isin(keep_numbers)]

    # Drop temporary column
    df.drop(columns=['Account_numeric'], inplace=True)

# Column to split data by
column_name = "Cost Centre"  # Ensure this matches exactly

# Function to sanitize sheet names (Excel limit: 31 characters)
def sanitize_sheet_name(name, max_length=31):
    sanitized = re.sub(r'[\/:*?"<>|]', '_', str(name))  # Replace invalid characters
    # Ensure the sheet name plus "_Stats" is no more than 31 characters
    stats_suffix = "_Stats"
    if len(sanitized) + len(stats_suffix) > max_length:
        sanitized = sanitized[:max_length - len(stats_suffix)]  # Keep space for "_Stats"
    return sanitized

# Output path for the final Excel file
output_excel_path = "/content/drive/My Drive/split_data_with_analysis.xlsx"

# Ensure column exists before proceeding
if column_name not in df.columns:
    print(f"Error: Column '{column_name}' not found in the dataset!")
else:
    with pd.ExcelWriter(output_excel_path, engine="xlsxwriter") as writer:
        for value, subset in df.groupby(column_name):
            sheet_name = sanitize_sheet_name(value)  # Ensure valid sheet name

            # Select only numeric columns for analysis
            numeric_cols = subset.select_dtypes(include=['number'])

            # Compute advanced statistics
            if not numeric_cols.empty:
                stats = numeric_cols.describe().T[['count', 'mean', 'min', 'max']]
                stats['sum'] = numeric_cols.sum()
                stats['std_dev'] = numeric_cols.std()
                stats['median'] = numeric_cols.median()
                stats['variance'] = numeric_cols.var()
                stats['iqr'] = numeric_cols.quantile(0.75) - numeric_cols.quantile(0.25)  # Interquartile Range
                stats.reset_index(inplace=True)  # Reset index to make it easy to read

                # Write subset data
                subset.to_excel(writer, sheet_name=sheet_name, index=False)

                # Write statistics sheet for each group
                stats.to_excel(writer, sheet_name=f"{sheet_name}_Stats", index=False)
            else:
                print(f"Warning: No numeric data to analyze for '{sheet_name}'.")

    print(f"Excel file '{output_excel_path}' created successfully with analysis sheets!")

# Save final filtered CSV file to Google Drive
output_csv_path_filtered = "/content/drive/My Drive/filtered_file.csv"
df.to_csv(output_csv_path_filtered, index=False)
print(f"Filtered CSV file saved to Google Drive at: {output_csv_path_filtered}")



Mounted at /content/drive


Downloading...
From: https://drive.google.com/uc?id=1pR8Jp7zOGV5OmWVa_5VuFBQlppK15Hi4
To: /content/downloaded_file.csv
100%|██████████| 19.2M/19.2M [00:00<00:00, 192MB/s]


File downloaded to: downloaded_file.csv
File read successfully!
'Cost Centre' column exists! Proceeding with splitting...
Excel file '/content/drive/My Drive/split_data_with_analysis.xlsx' created successfully with analysis sheets!
Filtered CSV file saved to Google Drive at: /content/drive/My Drive/filtered_file.csv
